In [1]:
import pandas as pd
import numpy as np

import twitter
import requests

from sqlalchemy import create_engine

import os
import yaml

import json
from IPython.display import display

ModuleNotFoundError: No module named 'twitter'

In [2]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [3]:
# здесь можно настроить поля, информацию по которым хотим получить для пользователя
# используем все доступные из списка документации

fields = [
    'created_at',
    'description',
#     'entities',
    'id',
    'location',
    'name',
    'pinned_tweet_id',
    'profile_image_url',
    'protected',
    'public_metrics', 
    'url',
    'username',
    'verified',
    'withheld'
]

# строим строку конфигурации полей, которую затем будем использовать при формировании запроса для обращения к API
fields_str = ','.join(fields)
user_fields = "user.fields={}".format(fields_str)

In [4]:
# для доступа к API также необходимо использовать bearer_token, полученный при регистрации приложения в Twitter


with open('../config/credentials.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

bearer_token = config['bearer_token']
headers = {"Authorization": "Bearer {}".format(bearer_token)}

NameError: name 'yaml' is not defined

In [5]:
def get_users_by_id(ids_list, user_fields, headers):
    # ids_list - список ИД пользователей, информацию для которых нужно получить
    # user_fields - список полей, которые нужно получить
    # headers - хидер для доступа к методам API
    
    # https://api.twitter.com/2/users?ids=... - API эндпойнт, который взвращает информацию о пользователях по списку их ИД
    # одновременно принимает максимум 100 ИД
    
    ids_str = ','.join(ids_list)
    ids = "ids={}".format(ids_str)
    get_users_by_id = "https://api.twitter.com/2/users?{}&{}".format(ids, user_fields)
    
    # ответ от эндпойнта с информацией о пользователях
    response = requests.request("GET", get_users_by_id, headers=headers)
    if response.status_code == 200:
        result = json.loads(response.content)['data']
        return result
    
    print(response.text)
    return []

In [6]:
df = pd.read_csv('../data/input/training_data.csv')

print(df.shape)
df.head(3)

(2797, 20)


,id,id_str,screen_name,location,description,url,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,8.160000e+17,"""815745789754417152""","""HoustonPokeMap""","""Houston, TX""","""Rare and strong PokŽmon in Houston, TX. See m...","""https://t.co/dnWuDbFRkt""",1291,0,10,"""Mon Jan 02 02:25:26 +0000 2017""",0,False,78554,"""en""","{\r ""created_at"": ""Sun Mar 12 15:44:04 +0...",True,False,False,"""Houston PokŽ Alert""",1
1,4.843621e+09,4843621225,kernyeahx,"Templeville town, MD, USA",From late 2014 Socium Marketplace will make sh...,NaN,1,349,0,2/1/2016 7:37,38,False,31,en,NaN,True,False,False,Keri Nelson,1
2,4.303727e+09,4303727112,mattlieberisbot,NaN,"Inspired by the smart, funny folks at @replyal...",https://t.co/P1e1o0m4KC,1086,0,14,Fri Nov 20 18:53:22 +0000 2015,0,False,713,en,"{'retweeted': False, 'is_quote_status': False,...",True,False,False,Matt Lieber Is Bot,1


In [7]:
# удаляем дубликаты пользователей по id_str

df = df.drop_duplicates(['id_str'], keep=False)
df.shape

(2331, 20)

In [8]:
# преобразуем формат ИД пользователя из scientific notation (e+09 etc.) в строку, чтобы осуществлять запросы к Twitter API

df['id_str'] = df['id'].map(int).map(str)

ids_for_request = df['id_str'].values.tolist()

In [9]:
# определяем, сколько итераций цикла нужно сделать, чтобы получить информацию о пользователях

n_requests = int(np.ceil(len(ids_for_request) / 100))
step = 100
n_requests, step

(24, 100)

In [10]:
# в цикле передаем методу, который получает информацию о пользователях, по 100 ИД итеративно, 
# пока не получим информацию для всех 2797 пользователей

n_steps = 0
users_results = []
for i in range(n_requests):
    ids_slice = ids_for_request[n_steps * step : (n_steps + 1) * step]
    slice_results = get_users_by_id(ids_slice, user_fields, headers)
    users_results.extend(slice_results)
    n_steps += 1
    
print('Info about {} users found'.format(len(users_results)))

Info about 2021 users found


In [11]:
df_users = pd.DataFrame(users_results)

print(df_users.shape)
df_users.head(2)

(2021, 12)


,name,username,description,verified,url,profile_image_url,public_metrics,created_at,id,protected,location,pinned_tweet_id
0,Matt Lieber Is Bot,mattlieberisbot,"Retired, for good this time • Inspired by the ...",False,https://t.co/oQKxig8kMF,https://pbs.twimg.com/profile_images/671119389...,"{'followers_count': 876, 'following_count': 0,...",2015-11-20T18:53:22.000Z,4303727112,False,NaN,NaN
1,single cell papers,sc_papers,,False,,https://abs.twimg.com/sticky/default_profile_i...,"{'followers_count': 325, 'following_count': 0,...",2015-02-25T20:11:25.000Z,3063139353,False,NaN,NaN


In [12]:
# парсим структуру public_metrics, чтобы получить данные о кол-ве друзей, твитов и т.д.

def parse_public_metrices(df):
    extension = df['public_metrics'].apply(pd.Series)
    ext_df = pd.concat([df, extension], axis=1).drop('public_metrics', axis=1)
    return ext_df

In [13]:
df_parsed_users = parse_public_metrices(df_users)

In [14]:
df_parsed_users['id']= df_parsed_users['id'].astype('str')

df_merged = df_parsed_users.merge(
    df[['id_str', 'bot']],
    left_on='id',
    right_on='id_str',
    how='inner'
).drop_duplicates(['id_str'], keep=False).drop(['id_str'], axis=1)

print('{} users found in Twitter'.format(df_merged.shape[0]))
df_merged.head(2)

2019 users found in Twitter


,name,username,description,verified,url,profile_image_url,created_at,id,protected,location,pinned_tweet_id,followers_count,following_count,tweet_count,listed_count,bot
0,Matt Lieber Is Bot,mattlieberisbot,"Retired, for good this time • Inspired by the ...",False,https://t.co/oQKxig8kMF,https://pbs.twimg.com/profile_images/671119389...,2015-11-20T18:53:22.000Z,4303727112,False,NaN,NaN,876,0,1182,16,1
1,single cell papers,sc_papers,,False,,https://abs.twimg.com/sticky/default_profile_i...,2015-02-25T20:11:25.000Z,3063139353,False,NaN,NaN,325,0,4031,12,1


In [15]:
# создаем папку под базу данных
if not os.path.exists('../data/db'):
    os.makedirs('../data/db')

# создаем базу для результатов и сохраняем данные о пользователях
DB_FOLDER = 'sqlite:///../data/db/'
DB_NAME = 'twitter_users.db'
DB_PATH = DB_FOLDER + DB_NAME

TABLE_NAME = 'users'

engine = create_engine(DB_PATH)
df_merged.to_sql(TABLE_NAME, engine, if_exists='replace', index=False)